# Projeto 1 - Ciência dos Dados

#### Nome: Andresa Bicudo

#### Nome: Leonardo Malta

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

### TO DO'S:

- Separar emojis (de palavras e de emojis)
- TABELA RELEVANTE ABSOLUTA + 1 / PALAVRAS RELEVANTES + PALAVRAS TOTAIS
- tabela relevante é de todas as palavras, precisamos delas individualmente

### Comentários:
- soma das palavras dos tweets relevantes = palavras_relevantes
- soma das palavras dos tweets irrelevantes = palavras_irrelevantes
- $P(Relevante) =$ palavras_relevantes/palavras_totais
- $P(Irrelevante) =$ palavras_irrelevantes/palavras_totais
- $P(Relevante) + P(Irrelevante) = 1$
- palavras_totais = palavras_relevantes + palavras_irrelevantes
- $P(tweet|Relevante) = $ DADO
- $P(tweet|Irrelevante) = $ DADO
- $P(Relevante|tweet) = $ CLASSIFICADOR
- $P(Irrelevante|tweet) = $ CLASSIFICADOR
- $P(Relevante|tweet) = P(tweet|Relevante)*P(Relevante)/P(tweet)$
- $P(Irrelevante|tweet) = P(tweet|Irrelevante)*P(Irrelevante)/P(tweet)$
- $P(tweet)$ é a probabilidade da frase ocorrer no espaço amostral total (palavras totais) - será desconsiderada porque ambas as probabilidades medidas pelo classificador, que determinarão se o tweet é relevante ou não, serão divididas por ela. Logo, o resultado qualitativo não se alterará, embora o resukltado quantitativo (percentual se alterará)

In [92]:
# Importanto bibliotecas

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

In [93]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\user\Documents\Insper\2° semestre\CDados\Projetos\Projeto1_Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [94]:
dados = pd.read_excel('Corona.xlsx')

In [95]:
dados.tail(5)

,Relevância,Mensagem
495,1,bahia\nclube coloca fazendão à disposição do g...
496,1,rt @clownhejhuff: 4 mortes por corona.... e o ...
497,1,pandemia de corona; cuide-se! #isolamento #is...
498,1,rt @tito_breno: primeiro caso de corona vírus ...
499,1,entrei nos casos com suspeita de corona em tub...


___
## Classificador automático de sentimento


#### Faça aqui uma descrição do seu tema e o que considerou como relevante ou não relevante na classificação dos tweets.

Em meio a essa grande pandemia do COVID-19 (Corona Vírus), é função Ministério da Saúde auxiliar a população com informações de prevenção e de cuidados cajo haja suspeita de contaminação, logo, tweets informativos são relevantes, uma vez que se pode evitar compartilhamento de "fake-news" e manter as recomendações verdadeiras. Ademais, também é essencial que o governo saiba da ocorrência de eventuais eventos que promovem aglomerações e de pessoas que banalizam a situação, a fim de tomar as medidas necessárias para conter o avanço dessa doença. Por fim, como o assunto está sendo extremamente comentado, muitos memes estão sendo feitos, logo, é preciso filtrá-los, recebendo a classificação de não relevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [96]:
# Transformando dados em variaveis qualitativas
dados.loc[:,'Mensagem'] = dados['Mensagem'].astype('category')
dados.loc[:,'Relevância'] = dados['Relevância'].astype('category')
dados.Relevância.cat.categories = ['Irrelevante', 'Relevante']
dados.head(5)

,Relevância,Mensagem
0,Relevante,rt @joaopauloz_: saber que algumas pessoas com...
1,Irrelevante,rt @thewayjane: se a campeã não sair daqui o c...
2,Relevante,"na espanha tá feio o corona, tenho medo de que..."
3,Irrelevante,"rt @george_fredson: dos mesmos criadores de ""a..."
4,Irrelevante,as primeiras que vão pega corona vírus aqui na...


In [97]:
# Separando dados em duas tabelas: Relevantes e Irrelevantes 
tweets_relevantes = dados.loc[dados['Relevância'] == 'Relevante', :]
tweets_irrelevantes = dados.loc[dados['Relevância'] == 'Irrelevante', :]

# Teste de separação de mensagem
tweets_irrelevantes.Mensagem[1]

'rt @thewayjane: se a campeã não sair daqui o corona já pode cancelar o bbb #bbb20 https://t.co/z4vvf9qn66'

In [98]:
# Criando listas com os índices dos Tweets relevante e irrelevantes
lista_indices_relevantes = []
lista_indices_irrelevantes = []

for i in range(len(dados.Mensagem)):
    if dados.Relevância[i] == 'Relevante':
        lista_indices_relevantes.append(dados.index[i])
    elif dados.Relevância[i] == 'Irrelevante':
        lista_indices_irrelevantes.append(dados.index[i])

In [99]:
# Função de limpeza que troca alguns sinais, menções e links por espaços, e coloca em uma lista.
# Link de consulta para fazer essa função: 
# https://stackoverflow.com/questions/54733828/remove-twitter-mentions-from-pandas-column

def cleanup(tweet):
    tweet_sem_mencoes = re.sub("@[A-Za-z0-9_]+","", tweet) # Retirando menções
    tweet_sem_links = re.sub(r'http\S+', '', tweet_sem_mencoes) # Retirando links
    punctuation = '[_ ! - . : ? ; @ , # / " * ( )]' # Sinais que serão substituidos por espaços
    pattern = re.compile(punctuation)
    tweet_sem_pontuacao = re.sub(pattern, ' ', tweet_sem_links) # Retirando sinais
    lista_tweet_semifinal = tweet_sem_pontuacao.split()
    # Removendo itens desnecessários para a classificação (retweets - 'rt', e artigos - 'a','o','as','os')\
    itens_desnecessarios = ['rt','a','o','as','os']  
    for item in itens_desnecessarios:
        for palavra in lista_tweet_semifinal:
            if palavra == item:
                lista_tweet_semifinal.remove(palavra)
            #if palavra == #emoji:
                
    lista_tweet_final = lista_tweet_semifinal
    return lista_tweet_final

# Teste do uso da função de limpeza
my_str = cleanup(tweets_irrelevantes.Mensagem[1])
print(my_str)

['se', 'campeã', 'não', 'sair', 'daqui', 'corona', 'já', 'pode', 'cancelar', 'bbb', 'bbb20']


In [100]:
# Colocando todas a palavras relevantes em uma lista, usando a lista de índices criadas anteriormente:
todas_palavras_relevantes = []
todas_palavras_irrelevantes = []

for i in range(len(lista_indices_relevantes)):
    todas_palavras_relevantes.append(cleanup(tweets_relevantes.Mensagem[lista_indices_relevantes[i]]))

for p in range(len(lista_indices_irrelevantes)):
    todas_palavras_irrelevantes.append(cleanup(tweets_irrelevantes.Mensagem[lista_indices_irrelevantes[p]]))

# Trasnformando uma lista de listas em uma única lista:
palavras_relevantes = []
palavras_irrelevantes = []

def Listas2lista(listadelistas):
    nova_lista = []
    for sublista in listadelistas:
        for item in sublista:
            nova_lista.append(item)
    return nova_lista

palavras_relevantes = Listas2lista(todas_palavras_relevantes)
palavras_irrelevantes = Listas2lista(todas_palavras_irrelevantes)

palvras_relevantes_data = pd.Series(palavras_relevantes)
palvras_irrelevantes_data = pd.Series(palavras_irrelevantes)

palavras_totais = palavras_relevantes + palavras_irrelevantes

print(len(palavras_totais))

8305


In [101]:
#Probabilidade de ser Relevante ou Irrelevante dentro de todas as palavras da base de treinamento
P_relevante = len(palavras_relevantes)/(len(palavras_totais))
P_irrelevante = len(palavras_irrelevantes)/(len(palavras_totais))

print('A probabilidade de qualque palavra ser relevante é de {0}%'.format((P_relevante*100)))
print('A probabilidade de qualque palavra ser irrelevante é de {0}%'.format(P_irrelevante*100))

A probabilidade de qualque palavra ser relevante é de 57.47140276941602%
A probabilidade de qualque palavra ser irrelevante é de 42.52859723058398%


In [102]:
# Tabelas de frequências absolutas:
tabela_relevantes_abs = palvras_relevantes_data.value_counts()
tabela_irrelevantes_abs = palvras_irrelevantes_data.value_counts()
tabela_irrelevantes_abs.head()

corona    252
de        102
vírus      87
que        78
eu         71
dtype: int64

In [103]:
# Tabelas de frequências relativas:
tabela_relevantes_rel = palvras_relevantes_data.value_counts(True)
tabela_irrelevantes_rel = palvras_irrelevantes_data.value_counts(True)
tabela_relevantes_rel.head()

corona    0.048188
de        0.036874
que       0.024513
e         0.022627
do        0.021580
dtype: float64

In [104]:
# Essa função calcula a Probabilidade de, dado o tweet, este ser relevante ou irrelevante:

palavras_sem_repetições = list(dict.fromkeys(palavras_totais))

def calcula_probabilidades(tweet, P_relevante, P_irrelevante):
    
    # Uso da função Cleanup para limpar o tweet e trasnformá-lo em uma lista de palavras
    tweet = cleanup(tweet)
    
    # Uso do Laplace smoothing
    P_tweet_dado_relevante = (((tabela_relevantes_abs[tweet])+1)/len(palavras_relevantes)+len(palavras_sem_repetições)).prod()
    P_tweet_dado_irrelevante = (((tabela_irrelevantes_abs[tweet])+1)/(len(palavras_irrelevantes)+len(palavras_sem_repetições))).prod()

    P_relevante_dado_tweet = P_tweet_dado_relevante  * P_relevante
    P_irrelevante_dado_tweet = P_tweet_dado_irrelevante * P_irrelevante
    
    probabilidades = [P_irrelevante_dado_tweet,P_relevante_dado_tweet]
    return probabilidades
    

In [105]:
# Teste do Classificador com Tweet aleatório
def Classificador(probabilidades):
    P_irrelevante_dado_tweet = probabilidades[0]
    P_relevante_dado_tweet = probabilidades[1]

    if (P_irrelevante_dado_tweet < P_relevante_dado_tweet):
        return 'Relevante'
    elif (P_irrelevante_dado_tweet > P_relevante_dado_tweet):
        return 'Irrelevante'
    
tweet_teste = "Para se previnir contra o Corona, lave as mãos com sabão"

teste = Classificador(calcula_probabilidades(tweet_teste, P_relevante, P_irrelevante))

print(teste)

Relevante


In [106]:
# Teste do Classificador com dados do Treinamento
resultado = []
for i in range(len(dados.Mensagem)):
    resultado.append(Classificador(calcula_probabilidades(dados.Mensagem[i], P_relevante, P_irrelevante)))

dados['Classificação Treinamento'] = resultado
dados.head(50)

,Relevância,Mensagem,Classificação Treinamento
0,Relevante,rt @joaopauloz_: saber que algumas pessoas com...,Relevante
1,Irrelevante,rt @thewayjane: se a campeã não sair daqui o c...,Relevante
2,Relevante,"na espanha tá feio o corona, tenho medo de que...",Relevante
3,Irrelevante,"rt @george_fredson: dos mesmos criadores de ""a...",Relevante
4,Irrelevante,as primeiras que vão pega corona vírus aqui na...,Relevante
5,Irrelevante,rt @anaalu1313: q ódio desse corona vairus put...,Relevante
6,Relevante,rt @lc_juh: em época de corona vírus trago diá...,Relevante
7,Irrelevante,os dente tudo podre nojento a pele escamando s...,Relevante
8,Relevante,rt @marcospiu_: corona vírus 🦠 \n\n-mais de 70...,Relevante
9,Irrelevante,@pedroneves_ @merinho_show @lucas_banks_ @dieg...,Relevante


In [107]:
x = dados['Classificação Treinamento'][1]
print(x)

Relevante


In [108]:
acertou = 0
errou = 0

for i in range(0,len(dados['Relevância'])):
    if dados['Relevância'][i] == dados['Classificação Treinamento'][i]:
        acertou += 1
    else:
        errou += 1
    
print(acertou)
print(errou)

250
250


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

In [109]:
#Testes para limpeza
tweet = tweets_irrelevantes.Mensagem[1]
tweet_sem_mencoes = re.sub("@[A-Za-z0-9]+","", tweet) # Retirando menções
tweet_sem_links = re.sub(r'http\S+', '', tweet_sem_mencoes) # Retirando links
punctuation = '[_ ! - . : ? ; @ ,]' # Sinais que serão substituidos por espaços
pattern = re.compile(punctuation)
tweet_sem_pontuacao = re.sub(pattern, ' ', tweet_sem_links) # Retirando sinais
lista_tweet_semifinal = tweet_sem_pontuacao.split()
    # Removendo itens desnecessários para a classificação (retweets - 'rt', e artigos - 'a','o','as','os')\
lista_tweet_semifinal.remove('rt')
lista_tweet_semifinal.remove('o')
lista_tweet_semifinal.remove('a')
lista_tweet_final = lista_tweet_sem_rt
print(lista_tweet_semifinal)

NameError: name 'lista_tweet_sem_rt' is not defined